In [1]:
from NCHelper import NCHelper
from datetime import datetime, timedelta
import xarray as xr
import time
import os
import shutil

In [18]:
# date_str = "2023110612"
date_str = "2023110800"
# dir_path = f"storage/{date_str}"
# if os.path.exists(dir_path):
#     shutil.rmtree(dir_path)
# os.mkdir(dir_path)

nch = NCHelper(
    f"http://182.16.248.173:8080/dods/INA-NWP/{date_str}/{date_str}-d01-asim"
)

reading dataset...
reading dataset done


c:\Users\falbas\miniconda3\Lib\site-packages\xarray\coding\times.py:167: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [19]:
time = nch.ds['time']
times = []
for t in time.values:
  dt = t.astype('M8[ms]').astype('O')
  times.append(dt.strftime("%Y%m%d%H"))

In [23]:
for i in range(0, len(times)-1, 8):
  print(f"{times[i]} {i}")

2023110800 0
2023110900 8
2023111000 16


In [ ]:
time = nch.ds['time']
times = []
for i in range(0, 2):
  dt = t.astype('M8[ms]').astype('O')
  times.append(dt.strftime("%Y%m%d%H"))

In [11]:
u = nch.ds["u"]
v = nch.ds["v"]
lat = nch.ds["lat"]
lon = nch.ds["lon"]
tc = nch.ds["tc"]
psfc = nch.ds["psfc"]

In [12]:
for i in range(len(times)):
    dir_path = f"storage/{times[i]}"
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.mkdir(dir_path)

    wind_dir = f"{dir_path}/wind"
    os.mkdir(wind_dir)
    nch.nc2asc(u[i][0], lat, lon, f"{wind_dir}/U")
    nch.nc2asc(v[i][0], lat, lon, f"{wind_dir}/V")



generating storage/2023110800/wind/U ascii grid...
output: d:/Kuliah/MBKM/PKKM BMKG/tes-netcdf/storage/2023110800/wind/U.asc
generating storage/2023110800/wind/V ascii grid...
output: d:/Kuliah/MBKM/PKKM BMKG/tes-netcdf/storage/2023110800/wind/V.asc
generating storage/2023110803/wind/U ascii grid...
output: d:/Kuliah/MBKM/PKKM BMKG/tes-netcdf/storage/2023110803/wind/U.asc
generating storage/2023110803/wind/V ascii grid...
output: d:/Kuliah/MBKM/PKKM BMKG/tes-netcdf/storage/2023110803/wind/V.asc
generating storage/2023110806/wind/U ascii grid...
output: d:/Kuliah/MBKM/PKKM BMKG/tes-netcdf/storage/2023110806/wind/U.asc
generating storage/2023110806/wind/V ascii grid...
output: d:/Kuliah/MBKM/PKKM BMKG/tes-netcdf/storage/2023110806/wind/V.asc
generating storage/2023110809/wind/U ascii grid...
output: d:/Kuliah/MBKM/PKKM BMKG/tes-netcdf/storage/2023110809/wind/U.asc
generating storage/2023110809/wind/V ascii grid...
output: d:/Kuliah/MBKM/PKKM BMKG/tes-netcdf/storage/2023110809/wind/V.asc
